<a href="https://colab.research.google.com/github/mlpbraga/sexism-detection-notebooks/blob/main/bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     |████████████████████████████████| 2.3MB 24.2MB/s 
     |████████████████████████████████| 901kB 48.0MB/s 
     |████████████████████████████████| 3.3MB 39.2MB/s 


In [23]:
import transformers
import torch
import math
import pandas as pd
from tqdm.notebook import trange, tqdm
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando: {device}")

Usando: cuda


In [20]:
class SentimentModel(torch.nn.Module):
    def __init__(self, pretrained_model):
        super().__init__()
        self.pretrained_model = pretrained_model
        self.dense1           = torch.nn.Linear(in_features=768 * 512, out_features=512)
        self.dense2           = torch.nn.Linear(in_features=512, out_features=1)
        self.dropout          = torch.nn.Dropout(0.2)
        self.relu             = torch.nn.ReLU()
        self.sigmoid          = torch.nn.Sigmoid()

    def forward(self, X):
        y_hat = self.pretrained_model(X['input_ids'], X['attention_mask'])
        y_hat = y_hat[0].flatten(start_dim=1)
        y_hat = self.dense1(y_hat)
        y_hat = self.relu(y_hat)
        y_hat = self.dropout(y_hat)
        y_hat = self.dense2(y_hat)
        y_hat = self.sigmoid(y_hat)
        return y_hat

In [21]:
class DisasterDataset(torch.utils.data.Dataset):
  def __init__(self, X_input_ids, X_attention_mask, y):
    self.X_input_ids = X_input_ids
    self.X_attention_mask = X_attention_mask
    self.y = y

  def __getitem__(self, index):
    _x_ii = self.X_input_ids[index]
    _x_am = self.X_attention_mask[index]
    _y = self.y[index]
    return _x_ii, _x_am, _y

  def __len__(self):
    return len(self.X_input_ids)

In [24]:
from sklearn.model_selection import train_test_split

labelad_data = pd.read_csv('https://raw.githubusercontent.com/mlpbraga/sexism-detection-notebooks/main/data/labeled-comments.csv')
dataset = labelad_data[['content','label']]

pretrained_model = transformers.AutoModel.from_pretrained('neuralmind/bert-base-portuguese-cased').to(device=device)
tokenizer = transformers.AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')

epochs = 5
X, y = list(dataset['content']), list(dataset['label'])

metrics = []
for i in range(0, 10):
  X_train_original, X_test_original, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

  for param in pretrained_model.parameters():
    param.requires_grad = False

  X_train = tokenizer(X_train_original,
                    padding='max_length',
                    truncation=True,
                    max_length=512,
                    return_tensors='pt').to(device)

  y_train = torch.Tensor(y_train).to(device=device).view(-1, 1)

  dd_train = DisasterDataset(X_train['input_ids'], X_train['attention_mask'], y_train)
  dl_train = torch.utils.data.DataLoader(dd_train, batch_size=64, shuffle=True)

  sm = SentimentModel(pretrained_model).to(device)
  sm.train()

  criterion = torch.nn.BCELoss()
  optimizer = torch.optim.Adam(sm.parameters(), lr=0.0001)

  results = []
  losses_train_mean = []
  for epoch in range(epochs):
    losses_train = []
    losses_val = []
    for batch in tqdm(dl_train):
      X_ii_b, X_am_b, y_b = batch
      X_ii_b = X_ii_b.to(device)
      X_am_b = X_am_b.to(device)
      y_b = y_b.to(device)

      optimizer.zero_grad()
      output = sm.forward({
          'input_ids': X_ii_b,
          'attention_mask': X_am_b
      })

      loss_train = criterion(output, y_b)
      losses_train.append(loss_train.item())
      loss_train.backward()
      optimizer.step()

    geral_loss = sum(losses_train)/len(dl_train)
    losses_train_mean.append(geral_loss)

    # test
    X_test_original, y_test   = list(test_data['content']), list(test_data['label'])
    X_test = tokenizer(X_test_original,
                      padding='max_length',
                      truncation=True,
                      max_length=512,
                      return_tensors='pt').to(device)
    y_test = torch.Tensor(y_test).to(device=device).view(-1, 1)


    dd_test = DisasterDataset(X_test['input_ids'], X_test['attention_mask'], y_test)
    dl_test = torch.utils.data.DataLoader(dd_test, batch_size=64, shuffle=True)

    all_results = []
    for batch in tqdm(dl_test):
      X_ii_b, X_am_b, y_b = batch
      X_ii_b = X_ii_b.to(device)
      X_am_b = X_am_b.to(device)
      y_b = y_b.to(device)
      sm.eval()
      with torch.no_grad():
        results = sm.forward({
          'input_ids': X_ii_b,
          'attention_mask': X_am_b
        }).cpu()
        for element in results:
          all_results.append(element)

    y_pred = []
    for result in all_results:
      value = 1 if result > 0.5 else 0
      y_pred.append(value)

    y_test_ = []
    for result in y_test:
      value = int(result)
      y_test_.append(value)

    metrics.append((precision_score(y_test_, y_pred),
                   recall_score(y_test_, y_pred),
                   f1_score(y_test_, y_pred)))


    print(f"Epoch: {epoch}, Loss: {round(geral_loss, 4)}")

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



Epoch: 0, Loss: 1.0528



Epoch: 1, Loss: 0.5198



Epoch: 2, Loss: 0.3786



Epoch: 3, Loss: 0.2727


...


Epoch: 0, Loss: 1.1168



Epoch: 1, Loss: 0.538



Epoch: 2, Loss: 0.3964



Epoch: 3, Loss: 0.2764


KeyboardInterrupt: ignored

In [ ]:
X_test_original, y_test   = list(test_data['content']), list(test_data['label'])
X_test = tokenizer(X_test_original,
                   padding='max_length',
                   truncation=True,
                   max_length=512,
                   return_tensors='pt').to(device)
y_test = torch.Tensor(y_test).to(device=device).view(-1, 1)


dd_test = DisasterDataset(X_test['input_ids'], X_test['attention_mask'], y_test)
dl_test = torch.utils.data.DataLoader(dd_test, batch_size=64, shuffle=True)

In [ ]:
train_size = math.ceil(labelad_data.shape[0] * 0.8)
val_size = math.floor(labelad_data.shape[0] * 0.)
test_size = math.floor(labelad_data.shape[0] * 0.1)

train_data = dataset[:train_size]
val_data = dataset[train_size:train_size+val_size]
test_data = dataset[train_size+val_size:train_size+val_size+test_size]

train_data = dataset[:train_size]
val_data = dataset[train_size:train_size+val_size]
test_data = dataset[train_size+val_size:train_size+val_size+test_size]

In [ ]:
X_train_original, y_train = list(train_data['content']), list(train_data['label'])
X_val_original, y_val     = list(val_data['content']), list(val_data['label'])

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
for param in pretrained_model.parameters():
    param.requires_grad = False

In [ ]:
X_train = tokenizer(X_train_original,
                    padding='max_length',
                    truncation=True,
                    max_length=512,
                    return_tensors='pt').to(device)
X_val = tokenizer(X_val_original,
                  padding='max_length',
                  truncation=True,
                  max_length=512,
                  return_tensors='pt').to(device)


In [ ]:
y_train = torch.Tensor(y_train).to(device=device).view(-1, 1)
y_val = torch.Tensor(y_val).to(device=device).view(-1, 1)

In [ ]:
dd_train = DisasterDataset(X_train['input_ids'], X_train['attention_mask'], y_train)
dl_train = torch.utils.data.DataLoader(dd_train, batch_size=64, shuffle=True)

dd_val = DisasterDataset(X_val['input_ids'], X_val['attention_mask'], y_val)
dl_val = torch.utils.data.DataLoader(dd_val, batch_size=64, shuffle=True)

In [ ]:
sm = SentimentModel(pretrained_model).to(device)
sm.train()
epochs = 3
criterion = torch.nn.BCELoss()
optimizer = torch.optim.SGD(sm.parameters(), lr=0.001)
# optimizer = torch.optim.Adam(sm.parameters())

In [ ]:
import tqdm
results = []
losses_train_mean = []
for epoch in range(epochs):
  losses_train = []
  losses_val = []
  for batch in tqdm.tqdm(dl_train):
    X_ii_b, X_am_b, y_b = batch
    X_ii_b = X_ii_b.to(device)
    X_am_b = X_am_b.to(device)
    y_b = y_b.to(device)

    optimizer.zero_grad()
    output = sm.forward({
        'input_ids': X_ii_b,
        'attention_mask': X_am_b
    })

    loss_train = criterion(output, y_b)
    losses_train.append(loss_train.item())
    loss_train.backward()

  # with torch.no_grad():
  #   sm.eval()
  #   losses_val_b = []
  #   for batch in tqdm.tqdm(dl_val):
  #     len_batch = len(batch)
  #     X_ii_b, X_am_b, y_b = batch
  #     X_ii_b = X_ii_b.to(device)
  #     X_am_b = X_am_b.to(device)
  #     y_b = y_b.to(device)
  #     output_val = sm.forward({
  #       'input_ids': X_ii_b,
  #       'attention_mask': X_am_b
  #     })
  #     loss_val = criterion(output_val, y_b)
  #     losses_val_b.append(loss_val.item())
  #   val_loss = sum(losses_val_b)/len_batch
  #   losses_val.append(val_loss)
  optimizer.step()

  geral_loss = sum(losses_train)/len(dl_train)
  val_loss = sum(losses_val)/len(dl_val)
  losses_train_mean.append(geral_loss)



  print(f"Epoch: {epoch}, Loss: {round(geral_loss, 4)}, Val: {round(val_loss,4)}")
  #results.append(show)




  0%|          | 0/45 [00:00<?, ?it/s]


  2%|▏         | 1/45 [00:02<01:56,  2.65s/it]


  4%|▍         | 2/45 [00:05<01:54,  2.65s/it]


  7%|▋         | 3/45 [00:07<01:51,  2.66s/it]


  9%|▉         | 4/45 [00:10<01:49,  2.66s/it]


 11%|█         | 5/45 [00:13<01:46,  2.67s/it]


 13%|█▎        | 6/45 [00:16<01:44,  2.68s/it]


 16%|█▌        | 7/45 [00:18<01:42,  2.69s/it]


 18%|█▊        | 8/45 [00:21<01:39,  2.70s/it]


 20%|██        | 9/45 [00:24<01:37,  2.71s/it]


 22%|██▏       | 10/45 [00:26<01:34,  2.71s/it]


 24%|██▍       | 11/45 [00:29<01:32,  2.72s/it]


 27%|██▋       | 12/45 [00:32<01:29,  2.72s/it]


 29%|██▉       | 13/45 [00:35<01:27,  2.72s/it]


 31%|███       | 14/45 [00:37<01:24,  2.73s/it]


 33%|███▎      | 15/45 [00:40<01:21,  2.73s/it]


 36%|███▌      | 16/45 [00:43<01:19,  2.73s/it]


 38%|███▊      | 17/45 [00:46<01:16,  2.75s/it]


 40%|████      | 18/45 [00:48<01:14,  2.76s/it]


 42%|████▏     | 19/45 [00:51<01:11,  2.77s/it]


 44%|████▍     

Epoch: 0, Loss: 0.693, Val: 0.2296





  2%|▏         | 1/45 [00:02<02:04,  2.82s/it]


  4%|▍         | 2/45 [00:05<02:01,  2.83s/it]


  7%|▋         | 3/45 [00:08<01:59,  2.84s/it]


  9%|▉         | 4/45 [00:11<01:57,  2.85s/it]


 11%|█         | 5/45 [00:14<01:54,  2.87s/it]


 13%|█▎        | 6/45 [00:17<01:52,  2.87s/it]


 16%|█▌        | 7/45 [00:20<01:49,  2.88s/it]


 18%|█▊        | 8/45 [00:23<01:46,  2.89s/it]


 20%|██        | 9/45 [00:25<01:44,  2.89s/it]


 22%|██▏       | 10/45 [00:28<01:41,  2.89s/it]


 24%|██▍       | 11/45 [00:31<01:38,  2.89s/it]


 27%|██▋       | 12/45 [00:34<01:35,  2.89s/it]


 29%|██▉       | 13/45 [00:37<01:32,  2.89s/it]


 31%|███       | 14/45 [00:40<01:29,  2.88s/it]


 33%|███▎      | 15/45 [00:43<01:26,  2.88s/it]


 36%|███▌      | 16/45 [00:46<01:23,  2.88s/it]


 38%|███▊      | 17/45 [00:48<01:20,  2.88s/it]


 40%|████      | 18/45 [00:51<01:17,  2.88s/it]


 42%|████▏     | 19/45 [00:54<01:14,  2.87s/it]


 44%|████▍     | 20/45 [00:57<01:11,  2.87s/it]


 47%|█

Epoch: 1, Loss: 0.6927, Val: 0.2303





  2%|▏         | 1/45 [00:02<02:04,  2.83s/it]


  4%|▍         | 2/45 [00:05<02:02,  2.84s/it]


  7%|▋         | 3/45 [00:08<01:59,  2.85s/it]


  9%|▉         | 4/45 [00:11<01:57,  2.86s/it]


 11%|█         | 5/45 [00:14<01:54,  2.86s/it]


 13%|█▎        | 6/45 [00:17<01:51,  2.86s/it]


 16%|█▌        | 7/45 [00:20<01:48,  2.86s/it]


 18%|█▊        | 8/45 [00:22<01:45,  2.86s/it]


 20%|██        | 9/45 [00:25<01:43,  2.87s/it]


 22%|██▏       | 10/45 [00:28<01:40,  2.86s/it]


 24%|██▍       | 11/45 [00:31<01:37,  2.87s/it]


 27%|██▋       | 12/45 [00:34<01:34,  2.87s/it]


 29%|██▉       | 13/45 [00:37<01:31,  2.87s/it]


 31%|███       | 14/45 [00:40<01:28,  2.87s/it]


 33%|███▎      | 15/45 [00:42<01:26,  2.87s/it]


 36%|███▌      | 16/45 [00:45<01:23,  2.87s/it]


 38%|███▊      | 17/45 [00:48<01:20,  2.87s/it]


 40%|████      | 18/45 [00:51<01:17,  2.87s/it]


 42%|████▏     | 19/45 [00:54<01:14,  2.87s/it]


 44%|████▍     | 20/45 [00:57<01:11,  2.87s/it]


 47%|█

Epoch: 2, Loss: 0.6896, Val: 0.2295


In [ ]:
for result in results:
  print(result)

In [ ]:
X_test_original, y_test   = list(test_data['content']), list(test_data['label'])
X_test = tokenizer(X_test_original,
                   padding='max_length',
                   truncation=True,
                   max_length=512,
                   return_tensors='pt').to(device)
y_test = torch.Tensor(y_test).to(device=device).view(-1, 1)


dd_test = DisasterDataset(X_test['input_ids'], X_test['attention_mask'], y_test)
dl_test = torch.utils.data.DataLoader(dd_test, batch_size=64, shuffle=True)

In [ ]:
all_results = []
for batch in tqdm.tqdm(dl_test):
  X_ii_b, X_am_b, y_b = batch
  X_ii_b = X_ii_b.to(device)
  X_am_b = X_am_b.to(device)
  y_b = y_b.to(device)
  sm.eval()
  with torch.no_grad():
    results = sm.forward({
      'input_ids': X_ii_b,
      'attention_mask': X_am_b
    }).cpu()
    for element in results:
      all_results.append(element)




  0%|          | 0/6 [00:00<?, ?it/s]


 17%|█▋        | 1/6 [00:02<00:13,  2.77s/it]


 33%|███▎      | 2/6 [00:05<00:11,  2.78s/it]


 50%|█████     | 3/6 [00:08<00:08,  2.78s/it]


 67%|██████▋   | 4/6 [00:11<00:05,  2.78s/it]


 83%|████████▎ | 5/6 [00:13<00:02,  2.80s/it]


100%|██████████| 6/6 [00:15<00:00,  2.57s/it]


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

y_pred = []
for result in all_results:
  value = 1 if result > 0.5 else 0
  y_pred.append(value)

y_test_ = []
for result in y_test:
  value = int(result)
  y_test_.append(value)

In [ ]:
precision_score(y_test_, y_pred)

0.46099290780141844

In [ ]:
recall_score(y_test_, y_pred)

0.39634146341463417

In [ ]:
f1_score(y_test_, y_pred)

0.42622950819672134

In [ ]:
print(classification_report(y_test_, y_pred))

              precision    recall  f1-score   support

           0       0.54      0.61      0.57       193
           1       0.46      0.40      0.43       164

    accuracy                           0.51       357
   macro avg       0.50      0.50      0.50       357
weighted avg       0.50      0.51      0.51       357

